# RCES Final Project

In [2]:
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/meredithkime/RCES_Final_Project/HEAD?labpath=final_project.ipynb)

SyntaxError: invalid syntax (3497253060.py, line 1)